In [4]:
%cd ..
root_dir = !pwd



/home/jabcross


In [2]:
%cd {root_dir[0]}
!pwd

exec_path = "/home/jabcross/repos/preesm-apps/SIFT/Code/Debug/bin/extract"
kernels = !grep 'iara.actor' 'test/Iara/05-cholesky/topology.mlir' |\
        grep -oE '@[a-zA-Z_]+'
if '@run' in kernels:
  kernels.remove('@run')

kernels = [k[1:] for k in kernels]

print(kernels)



In [41]:
# profile
# Using sudo to be able to get all core information.
# Be careful with the perf.data, it may have personal information.

%cd {root_dir[0]}
%cd profile
%mkdir work
%cd work

dir = !dirname {exec_path}
print('dir: ', dir[0])
%cd {dir[0]}

!pwd
command = f'perf record \
               -e cycles \
               -e sched:sched_switch \
               --switch-events \
               --sample-cpu \
               --call-graph dwarf \
               -a -g -T -F 99 ./extract && \
               chown "$USER:$USER" perf.data && \
               chmod ugo+rw perf.data'
# !run0 sh -x -c '{command}'

!mv perf.data {root_dir[0]}/profile/work

!pwd



In [1]:
workdir = "/home/jabcross/repos/iara/profile/work"
%cd $workdir

!perf data convert --to-json perf-data.json

import json
import fileinput

with open("perf-data.json") as file:
  data = json.load(file)

print(len(data))


/home/jabcross/repos/iara/profile/work
Output file exists. Use --force to overwrite it.
3


In [39]:
import pandas as pd

df = pd.json_normalize(data['samples'])

df = df[(df['comm']=='extract') & (len(df['callchain']) > 0)]

df.to_csv('extract.csv')





In [35]:
# get only lines where callchain is not  empty

import numpy as np

has_callchain = df[df['callchain'].apply(lambda x: len(x) > 0)]

has_callchain



,timestamp,pid,tid,cpu,comm,callchain,prev_comm,prev_pid,prev_prio,prev_state,next_comm,next_pid,next_prio
158,118567340780477,795984,795986,1,extract,"[{'ip': '0xffffffffa45b0cc3', 'symbol': 'mas_w...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
164,118567341118682,795984,795987,2,extract,"[{'ip': '0xffffffffa46665ab', 'symbol': '_raw_...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
169,118567342861740,795984,795988,3,extract,"[{'ip': '0xffffffffa45cec0b', 'symbol': 'clear...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
170,118567343521333,795984,795986,1,extract,"[{'ip': '0xffffffffa45cec0b', 'symbol': 'clear...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
171,118567347164207,795984,795984,0,extract,"[{'ip': '0xffffffffa4801280', 'symbol': 'asm_e...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28741,118603859835455,795984,795987,2,extract,"[{'ip': '0xffffffffa42ae6c1', 'symbol': 'dw_re...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
28825,118603970283700,795984,795984,0,extract,"[{'ip': '0xffffffffa3c55b25', 'symbol': 'inode...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
28932,118604110872409,795984,795986,1,extract,"[{'ip': '0xffffffffa3996cfd', 'symbol': 'timek...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
28953,118604134299781,795984,795987,2,extract,"[{'ip': '0xffffffffa42ae6c1', 'symbol': 'dw_re...",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
# get important symbol names from topology

actortype_names = !grep 'actortype .* {' /home/jabcross/repos/iara/test/Iara/08-SIFT/topology.dif \
    | sed -E 's/^.*actortype (.*) \{$.*/\1/'

actortype_names

 

['row_filter_transpose2x_1',
 'row_filter_transpose2x_2',
 'BarrierTranspose2x_1',
 'BarrierTranspose2x_2',
 'row_filter_transpose_1',
 'row_filter_transpose_2',
 'BarrierTranspose_1',
 'BarrierTranspose_2',
 'compute_gaussian_coefs',
 'extract_descriptor',
 'detect_keypoints',
 'ITERATOR_detect_keypoints',
 'BdRot_BROADCAST',
 'BdGrd_BROADCAST',
 'build_dog_pyr',
 'ITERATOR_build_dog_pyr',
 'build_grd_rot_pyr',
 'ITERATOR_build_grd_rot_pyr',
 'to_float',
 'BdGpyr_BROADCAST',
 'MERGE_gpyr',
 'BdFloatImg_BROADCAST',
 'upsample2x',
 'downsample2xN',
 'BdCoefs_BROADCAST',
 'BdSizes_BROADCAST',
 'BdBlurUp2x_BROADCAST',
 'BdBlurDown2x1_BROADCAST',
 'seq_blur1',
 'downsample2x1',
 'seq_blurN',
 'BdBlurDown2xN_BROADCAST',
 'counterGpyrLayer',
 'BdCounterGpyr_BROADCAST',
 'counterOctaveDownN',
 'BdGT_BROADCAST',
 'BdGT_2x_BROADCAST',
 'BdBlurred1_BROADCAST',
 'BdBlurredN_BROADCAST',
 'BdOctaveDown_BROADCAST',
 'MERGE_keypoints',
 'BdCounterGpyrXOctave_BROADCAST',
 'BdDoG_BROADCAST',
 'BarrierC

In [113]:
%cd /home/jabcross/repos/preesm-apps/SIFT/Code/Release/bin
!perf script -i perf.data > tmp.txt

perf_output = !cat tmp.txt

# perf_output

state_log = []

from itertools import groupby

cores = ['[000]','[001]','[002]','[003]']
samples = [list(sub) for i, sub in groupby(perf_output, key=bool) if i]
print(len(samples))

for sample in samples:
  header = sample[0].split()
  sample_data = header[4:]

  command, pid, core, timestamp  = header[:4]
  if core not in cores:
    continue
  # print(sample_data)
  # print(command, pid, core, timestamp)
  call_stack = ','.join(map(lambda x: x.split()[1], sample[1:]))
  if "core/cycles" in sample_data[1]:
    symbols_in_core = []
    if command == 'extract':
      for name in actortype_names:
        if name in call_stack:
          symbols_in_core.append(name)
    state_log.append((timestamp, core, symbols_in_core))
  if command == 'extract' and 'sched:sched_switch' in sample[0]:
    state_log.append((timestamp, core, []))

print(len([i for i in state_log if not i[2] ]))
state_log

/home/jabcross/repos/preesm-apps/SIFT/Code/Release/bin
1 out of order events recorded.
132925
3341


[('15825.173787:', '[000]', []),
 ('15825.173805:', '[000]', []),
 ('15825.173878:', '[000]', []),
 ('15825.173889:', '[000]', []),
 ('15825.173926:', '[001]', []),
 ('15825.173940:', '[001]', []),
 ('15825.174000:', '[002]', []),
 ('15825.174011:', '[002]', []),
 ('15825.174133:', '[003]', []),
 ('15825.174147:', '[003]', []),
 ('15825.177885:', '[000]', []),
 ('15825.178537:', '[000]', []),
 ('15825.178775:', '[000]', []),
 ('15825.179026:', '[000]', []),
 ('15825.179961:', '[002]', []),
 ('15825.179978:', '[003]', []),
 ('15825.180051:', '[002]', []),
 ('15825.180066:', '[003]', []),
 ('15825.180143:', '[002]', []),
 ('15825.180152:', '[003]', []),
 ('15825.180339:', '[002]', []),
 ('15825.180347:', '[003]', []),
 ('15825.180794:', '[003]', []),
 ('15825.181471:', '[003]', []),
 ('15825.182084:', '[003]', []),
 ('15825.182626:', '[003]', []),
 ('15825.182745:', '[000]', []),
 ('15825.183088:', '[003]', []),
 ('15825.183524:', '[001]', []),
 ('15825.183593:', '[002]', []),
 ('15825.1

In [124]:
start_time = float(state_log[0][0][:-1])
end_time = start_time
total_core_usage = {core: 0.0 for core in cores}
last_funcs_in_core = {core: [] for core in cores}
last_time_core_turned_on = {core: None for core in cores}
running_cores = set()
is_function_running_in_core = {core: {func for func in actortype_names} for core in cores}
total_func_runtime = {func: 0.0 for func in actortype_names}
func_bottleneckiness = {func: 0.0 for func in actortype_names}
num_cores = len(cores)

for timestamp, core, funcs in state_log:
  curr_time = float(timestamp[:-1])
  start_time = min(start_time, curr_time)
  end_time = max(start_time, curr_time)
  timestep = 0.0

  # core has kept running
  if last_time_core_turned_on[core] and funcs:
    timestep = curr_time - last_time_core_turned_on[core]
    total_core_usage[core] += timestep
    running_cores.add(core)
    last_time_core_turned_on[core] = curr_time
    is_function_running_in_core[core] = set()
    for func in funcs:
      total_func_runtime[func] += timestep
      func_bottleneckiness[func] += timestep * num_cores / len(running_cores)
      is_function_running_in_core[core].add(func)

  # core has switched off
  elif not funcs:
    last_funcs_in_core[core] = set()
    running_cores.remove(core)
    last_time_core_turned_on[core] = None
    
  # corre has switched on
  elif not last_time_core_turned_on[core] and funcs:
    running_cores.add(core)
    last_time_core_turned_on[core] = curr_time

  is_function_running_in_core[core] = set()
  for func in funcs:
    is_function_running_in_core[core].add(func)
  last_funcs_in_core[core] = funcs

print(total_core_usage)

print(f"{sum(total_core_usage.values())=}")
print(f"{end_time-start_time=}")

{'[000]': 30.652097999991383, '[001]': 33.33331600000747, '[002]': 32.945856999984244, '[003]': 27.99042700001519}
sum(total_core_usage.values())=124.92169799999829
end_time-start_time=34.73992999999973
